In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import jaccard_score
import os
from sklearn.svm import l1_min_c
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score as cvs
import sys

In [3]:
#os.chdir("C:\\Users\\tbrej\\Dataproject")

In [5]:
meth_even = pd.read_csv("processed_data\combined_data\split_with_background\\2mers_extend200\combined_2mers_meth_even_with_background.tsv", sep="\t")
meth_odd = pd.read_csv("processed_data\combined_data\split_with_background\\2mers_extend200\combined_2mers_meth_odd_with_background.tsv", sep="\t")
unmeth_even = pd.read_csv("processed_data\combined_data\split_with_background\\2mers_extend200\combined_2mers_unmeth_even_with_background.tsv", sep="\t")
unmeth_odd = pd.read_csv("processed_data\combined_data\split_with_background\\2mers_extend200\combined_2mers_unmeth_odd_with_background.tsv", sep="\t")


meth_even_healthy = meth_even.loc[meth_even["cancer"]=="Healthy"]

meth_even_unhealthy = meth_even.loc[meth_even["cancer"]!="Healthy"]

meth_odd_healthy = meth_odd.loc[meth_odd["cancer"]=="Healthy"]

meth_odd_unhealthy = meth_odd.loc[meth_odd["cancer"]!="Healthy"]

unmeth_even_healthy = unmeth_even.loc[unmeth_even["cancer"]=="Healthy"]

unmeth_even_unhealthy = unmeth_even.loc[unmeth_even["cancer"]!="Healthy"]

unmeth_odd_healthy = unmeth_odd.loc[unmeth_odd["cancer"]=="Healthy"]

unmeth_odd_unhealthy = unmeth_odd.loc[unmeth_odd["cancer"]!="Healthy"]

meth_even_healthy = meth_even_healthy.drop(306)

meth_odd_healthy = meth_odd_healthy.drop(306)

unmeth_even_healthy = unmeth_even_healthy.drop(306)

unmeth_odd_healthy = unmeth_odd_healthy.drop(306)




#index to get get half the healthy 
tr = np.shape(meth_odd_healthy)[0]//2

# Training data 

tr_meth_even_healthy = meth_even_healthy[0:tr]

tr_meth_odd_healthy = meth_odd_healthy[0:tr]

tr_unmeth_even_healthy = unmeth_even_healthy[0:tr]

tr_unmeth_odd_healthy = unmeth_odd_healthy[0:tr]

# Test data 

test_meth_even_healthy = meth_even_healthy[tr:]

test_meth_odd_healthy = meth_odd_healthy[tr:]

test_unmeth_even_healthy = unmeth_even_healthy[tr:]

test_unmeth_odd_healthy = unmeth_odd_healthy[tr:]


# Matrices

train_matrix = [tr_meth_even_healthy, tr_meth_odd_healthy, tr_unmeth_even_healthy, tr_unmeth_odd_healthy]

pred_matrix = [test_meth_odd_healthy, test_meth_even_healthy, test_unmeth_even_healthy, test_unmeth_odd_healthy, meth_even_unhealthy, meth_odd_unhealthy, unmeth_even_unhealthy, unmeth_odd_unhealthy]

train_matrix = pd.concat(train_matrix) 
train_matrix = train_matrix.iloc[: , :-1]

pred_matrix = pd.concat(pred_matrix) 
pred_matrix = pred_matrix.iloc[: , :-1]



train_targets = [0] * np.shape(tr_meth_even_healthy)[0] + [0] * np.shape(tr_meth_odd_healthy)[0] +  [1] * np.shape(tr_unmeth_even_healthy)[0] + [1] * np.shape(tr_unmeth_odd_healthy)[0] 

pred_targets = [0] * np.shape(test_meth_even_healthy)[0] + [0] * np.shape(test_meth_odd_healthy)[0] +  [1] * np.shape(test_unmeth_even_healthy)[0] + [1] * np.shape(test_unmeth_odd_healthy)[0] + [0] * np.shape(meth_even_unhealthy)[0] +   [0] * np.shape(meth_odd_unhealthy)[0] +  [1] * np.shape(unmeth_even_unhealthy)[0] +  [1] * np.shape(unmeth_odd_unhealthy)[0]


In [12]:
def pca(train_data, test_data, n_comp): 
    dim_reduction = PCA(n_components= n_comp)
    train_fit = dim_reduction.fit_transform(train_data)
    test_fit = dim_reduction.transform(test_data)
    return [train_fit, test_fit]



In [20]:
train_fit, test_fit = pca(train_matrix, pred_matrix, 16) 

cs = l1_min_c(train_fit, train_targets, loss = 'log') * np.logspace(0,10,16)

empty_model = LR(penalty = 'l1',solver='liblinear',tol=1e-6,max_iter=1000,warm_start=True,intercept_scaling=10000.0)

coefs_ = []
for c in cs:
    empty_model.set_params(C=c)
    empty_model.fit(train_fit, train_targets)
    coefs_.append(empty_model.coef_.ravel().copy())
coefs_ = np.array(coefs_)

preds = empty_model.predict_proba(test_fit)
np.set_printoptions(threshold=sys.maxsize)


score = cvs(empty_model, preds, pred_targets)
print(score.mean())
print(preds)
#train_preds, test_preds, coef, score = pen_lasso(train_fit, train_targets, test_fit)

#LR_preds = classification(train_preds, train_targets, test_preds)

#LR_preds






0.9539182517605193
[[1.00000000e+00 1.05100148e-19]
 [1.00000000e+00 4.55785635e-14]
 [1.00000000e+00 2.00498440e-26]
 [1.00000000e+00 4.94881172e-15]
 [1.00000000e+00 5.48096145e-28]
 [1.00000000e+00 3.55205141e-12]
 [1.00000000e+00 1.53466371e-16]
 [1.00000000e+00 4.89544756e-25]
 [1.00000000e+00 6.35377896e-21]
 [1.00000000e+00 3.34199404e-19]
 [1.00000000e+00 1.38177300e-24]
 [1.00000000e+00 1.99513411e-23]
 [1.00000000e+00 1.30501654e-28]
 [1.00000000e+00 4.18604273e-24]
 [1.00000000e+00 9.73851161e-23]
 [1.00000000e+00 5.66913193e-15]
 [1.00000000e+00 6.68993311e-20]
 [1.00000000e+00 8.64437751e-21]
 [1.00000000e+00 5.01738868e-25]
 [1.00000000e+00 1.78298802e-13]
 [1.00000000e+00 2.70894860e-17]
 [1.00000000e+00 6.80154579e-17]
 [1.00000000e+00 1.68727089e-21]
 [1.00000000e+00 3.50829656e-14]
 [1.00000000e+00 5.31746120e-22]
 [1.00000000e+00 1.52695055e-23]
 [1.00000000e+00 7.20707493e-12]
 [1.00000000e+00 4.77318597e-22]
 [1.00000000e+00 2.59646472e-27]
 [1.00000000e+00 8.65994

c:\Users\tbrej\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
